In [1]:
# Authenticate user
import spotipy
import spotipy.util as util

def get_token(scope):
    token = util.prompt_for_user_token(
            username='',
            scope=scope,
            client_id='e74c52988f6d4bcebb36970a423d348d',
            client_secret='0edc87deae1a4611a97b6cebef262136',
            redirect_uri='http://localhost:8888/callback')
    return token
token = get_token(scope='user-top-read')
if token:
    sp = spotipy.Spotify(auth=token)

In [2]:
# Experiment
# Get user's profile information
profile = sp.current_user()

In [21]:
print [u'\u2019']

[u'\u2019']


In [13]:
# Experiment
# Get user's top artists
# Part 1: Function
def get_users_top_artists(limit=20, time_range='long_term'):
    results = sp.current_user_top_artists(limit=limit, offset=0, time_range=time_range)
    return results['items']

In [16]:
# Part 2: Print
for artist in get_users_top_artists(time_range='long_term'):
    print artist['name']

Gramatik
Jon Hopkins
Pretty Lights
Onra
Lusine
The Go! Team
18 Carat Affair
Max Richter
Lone
Piano Chillout Magic
Eluvium
Two Door Cinema Club
Daft Punk
Oneohtrix Point Never
Caravan Palace
Starcadian
Aphex Twin
ODESZA
Goldmund
Emancipator


In [158]:
# Experiment
# Part 1: Get related artists
def get_related_artists(artist):
    return sp.artist_related_artists(artist[u'id'])

In [157]:
related_artists = {}
for artist in get_users_top_artists():
    related_artists[artist['name']] = get_related_artists(artist)

In [159]:
# Part 2: Print artist and similar artists
for key, val in related_artists.iteritems():
    artist_name = key
    similar_artists = val['artists']
    similar_artist_names = [artist['name'] for artist in similar_artists]
    print artist_name
    print similar_artist_names

Daft Punk
[u'Justice', u'Cassius', u'The Chemical Brothers', u'Basement Jaxx', u'Kavinsky', u'Calvin Harris', u'Vitalic', u'MSTRKRFT', u'David Guetta', u'Gorillaz', u'Mr. Oizo', u'Digitalism', u'The Bloody Beetroots', u'Mylo', u'Moby', u'Soulwax', u'deadmau5', u'R\xf6yksopp', u'The Prodigy', u'Fatboy Slim']
Big Black Delta
[u'Austra', u'White Sea', u'Class Actress', u'Young Galaxy', u'Superhumanoids', u'MNDR', u'WIN WIN', u'Twin Shadow', u'Glasser', u'Exitmusic', u'Fol Chen', u'Zambri', u'Chromatics', u'Born Gold', u'Gardens & Villa', u'Active Child', u'Young Magic', u'Small Black', u'Hooray For Earth', u'I Break Horses']
Beat Connection
[u'Chad Valley', u'Kisses', u'Brothertiger', u'Lemonade', u'Millionyoung', u'Work Drugs', u'Tanlines', u'Summer Heart', u'Teen Daze', u'Blackbird Blackbird', u'Small Black', u'Javelin', u'Craft Spells', u'Porcelain Raft', u'Keep Shelly In Athens', u'Memory Tapes', u'Body Language', u'Clive Tanaka y su orquesta', u'Hooray For Earth', u'Neon Indian']
The

In [5]:
# Experiment
# Part 1: Get artist remixes
def get_remixes(artist, limit=50, offset=0):
    # Returns the query searching for artist + remix
    name = artist['name']
    
    def search_spotify_tracks(query):
        results = sp.search(q=query, type='track', limit=limit, offset=offset)
        tracks = [track for track in results['tracks']['items']]
        return tracks
     
    remix_tracks = search_spotify_tracks('{} remix'.format(name))
    return remix_tracks

def get_all_remixes(artist):
    all_remixes = []
    offset = 0
    limit = 50
    while True:
        remixes = get_remixes(artist, limit=limit, offset=offset)
        if not remixes:
            break
        all_remixes.extend(remixes)
        offset += limit
    return all_remixes

def get_tracks_remixed_by(artist):
    # filter only tracks the artist has remixed
    def artist_uris(track):
        return [a['uri'] for a in track['artists']]
    
    #remixes = [remix for remix in get_remixes(artist)
    #          if artist['uri'] not in remix['artists'][0]['uri']]
    remixes = [remix for remix in get_remixes(artist)
              if artist['name'] in remix['name']]
    return remixes
    
def get_tracks_others_remixed(artist):
    # filter tracks by the artist others have remixed 
    def artist_uris(track):
        return [a['uri'] for a in track['artists']]
    
    remixes = [remix for remix in get_remixes(artist)
               if artist['uri'] in remix['artists'][0]['uri']]
    return remixes

In [ ]:
# Part 2: Print artist remixes 
# TODO: Add boards of canada test, should find 4
for artist in get_users_top_artists():
    remixes = get_all_remixes(artist)
    print artist['name'] ,': ', [t['name'] for t in remixes]

In [137]:
# Test
# Find all remixes
boc_uri = 'spotify:artist:2VAvhf61GgLYmC6C8anyX1'
artist = sp.artist(boc_uri)
remixes = get_all_remixes(artist)
print artist['name']
print [t['name'] for t in remixes]

Boards of Canada
[u'Last walk around mirror lake - Boards of Canada remix', u'Broken Drum - Remixed By Boards Of Canada', u'Mr Mistake (Boards Of Canada Remix)', u'Dayvan Cowboy (Odd Nosdam Remix)', u'Dead Dogs Two (Boards Of Canada Remix) - mixed', u'Dead Dogs Two - Boards of Canada remix', u'Kid For Today (Stereolab Mix)', u'Dead Dogs Two (Boards Of Canada Remix) - mixed', u'Poppy Seed (Boards Of Canada Remix)', u'Dayvan Cowboy (Remix)', u'Last walk around mirror lake - Boards of Canada remix', u'Sandsings (remixed by Boards of Canada)', u'An Eagle In Your Mind (Consumed And Regurgitated By The P.B.O.Ster!)', u'Poppy Seed (Reprise) (Boards Of Canada Remix)']


In [147]:
# Test
# Find all artist tracks remixed by others
boc_uri = 'spotify:artist:4tZwfgrHOc3mvqYlEYSvVi'
artist = sp.artist(boc_uri)
remixes = get_tracks_others_remixed(artist)
print artist['name']
print [t['artists'][0]['name'] + '-' + t['name'] for t in remixes]

Daft Punk
[u'Daft Punk-Derezzed - Remixed by The Glitch Mob', u'Daft Punk-Get Lucky - Daft Punk Remix', u'Daft Punk-Aerodynamic (Daft Punk Remix)', u'Daft Punk-Fall - Remixed by M83 VS Big Black Delta', u'Daft Punk-The Grid - Remixed by The Crystal Method', u'Daft Punk-Derezzed - Remixed by Avicii', u'Daft Punk-The Son of Flynn - Remixed by Moby', u'Daft Punk-Adagio for TRON - Remixed by Teddybears', u'Daft Punk-End of Line - Remixed by Photek', u'Daft Punk-The Son of Flynn - Remixed by Ki:Theory', u'Daft Punk-Solar Sailer - Remixed by Pretty Lights', u'Daft Punk-End of Line - Remixed by Boys Noize', u'Daft Punk-Rinzler - Remixed by Kaskade', u'Daft Punk-C.L.U. - Remixed by Paul Oakenfold', u'Daft Punk-Encom Part 2 - Remixed by Com Truise', u'Daft Punk-TRON Legacy (End Titles) - Remixed by Sander Kleinenberg', u'Daft Punk-Arena - Remixed by The Japanese Popstars', u'Daft Punk-Robot Rock - Soulwax Remix', u'Daft Punk-Human After All - SebastiAn Remix', u'Daft Punk-Harder Better Faster S

In [236]:
# Test
# Find all tracks remixed by artist
boc_uri = 'spotify:artist:43mWhBXSflupNLuNjM5vff'
artist = sp.artist(boc_uri)
remixes = get_tracks_remixed_by(artist)
print artist['name']
print [t['artists'][0]['name'] + '-' + t['name'] for t in remixes]

Soulwax
[u'Tame Impala-Let It Happen - Soulwax Remix', u'Jungle-Julia (Soulwax Remix)', u"The Rolling Stones-You Can't Always Get What You Want (Soulwax Remix) - Remix", u'Daft Punk-Robot Rock - Soulwax Remix', u'Gorillaz-DARE - Soulwax Remix', u'Gorillaz-DARE - Soulwax Remix', u'Joe Goddard-Gabriel - Soulwax Remix', u'Hot Chip-Huarache Lights - Soulwax Remix', u'Justice-Phantom Pt II - Soulwax Remix', u'Muse-Muscle Museum [Soulwax Remix]', u'Hot Chip-Huarache Lights - Soulwax Remix', u'LCD Soundsystem-Get Innocuous! - Soulwax Remix', u'LCD Soundsystem-You Wanted A Hit - Soulwax Remix', u'Gorillaz-DARE - Soulwax Remix', u'Ladytron-Seventeen - Soulwax Mix', u'Hot Chip-Huarache Lights - Soulwax Remix', u'Late of the Pier-Best in The Class - Soulwax Remix', u'LCD Soundsystem-Get Innocuous! - Soulwax Remix', u'Ladytron-Seventeen - Soulwax Mix', u'Justice-Phantom Pt II - Soulwax Remix', u'Zita Swoon-My Bond With You and Your Planet: Disoc! - Soulwax Remix']


In [238]:
t['name']

u'My Bond With You and Your Planet: Disoc! - Soulwax Remix'

In [228]:
class Test(dict):
    
    def __init__(self, a):
        dict.__init__(self, a)
    
    def cool_method(self):
        print 'hello world'
        
    def __str__(self):
        return ' '.join(self.keys())
a['awefaew'] = 2
b = Test(a)

In [229]:
b.cool_method()

hello world


In [230]:
a.keys()

['awefaew']

In [231]:
print b

awefaew


In [198]:
b.awefaew

AttributeError: 'Test' object has no attribute 'awefaew'

In [281]:
import os
import sys
sys.path.append('/Users/alexanderlitven/Projects/remix_finder/src')

In [282]:
from spotify_adaptors import User, Artist, Track
reload(spotify_adaptors)
u = User()

In [283]:
top_artists = u.get_top_artists()

In [284]:
print([str(item) for item in top_artists])

['The Go! Team', 'Daft Punk', 'Starcadian', 'Gramatik', 'Pretty Lights', 'Ulises Conti', 'The New Division', 'Vulfpeck', 'Battle Tapes', 'Moby', 'Big Black Delta', 'The Polish Ambassador', 'Death In Vegas', 'Lone', 'The War On Drugs', 'Low Motion Disco', 'Nils Frahm', '18 Carat Affair', 'Cash+David', "The La's"]


In [280]:
for artist in top_artists:
    print artist, "top tracks:"
    top_tracks = artist.get_top_tracks()
    print [str(t) for t in top_tracks]

The Go! Team 's top tracks:
['The Go! Team - Apollo Throwdown (Star Slinger Remix)', 'The Go! Team - Huddle Formation', "The Go! Team - Everyone's A Vip To Someone", 'The Go! Team - Ladyflash', 'The Go! Team - Get It Together', 'The Go! Team - Grip Like A Vice', 'The Go! Team - Feel Good By Numbers', 'The Go! Team - The Power Is On', 'The Go! Team - Junior Kickstart', 'The Go! Team - Panther Dash']
Daft Punk 's top tracks:
['Daft Punk - Get Lucky - Radio Edit', 'Daft Punk - Get Lucky', 'Daft Punk - Instant Crush', 'Daft Punk - One More Time', 'Daft Punk - Lose Yourself to Dance', 'Daft Punk - Around The World', 'Daft Punk - Harder Better Faster Stronger', "Daft Punk - Doin' it Right", 'Daft Punk - Give Life Back to Music', 'Daft Punk - Giorgio by Moroder']
Starcadian 's top tracks:
['Starcadian - He^rt', 'Starcadian - Chinatown', 'Starcadian - He^Rt (Album Mix)', 'Starcadian - Ultralove', 'Starcadian - Lovetop', 'Starcadian - Sgt. Tagowski', 'Starcadian - Dance or Die', 'Starcadian - R

UnicodeEncodeError: 'ascii' codec can't encode character u'\xd3' in position 0: ordinal not in range(128)

In [277]:
top_artists

[{u'external_urls': {u'spotify': u'https://open.spotify.com/artist/6veTV9sF06FBf2KN0xAdvo'},
  u'followers': {u'href': None, u'total': None},
  u'genres': [],
  u'href': u'https://api.spotify.com/v1/artists/6veTV9sF06FBf2KN0xAdvo',
  u'id': u'6veTV9sF06FBf2KN0xAdvo',
  u'images': [{u'height': 1000,
    u'url': u'https://i.scdn.co/image/733fa2d1eab63b9d3741ad640e933e13d23a8b4d',
    u'width': 1000},
   {u'height': 640,
    u'url': u'https://i.scdn.co/image/ea86120b7ef58fe5d8f9b426102821b77b2314a6',
    u'width': 640},
   {u'height': 200,
    u'url': u'https://i.scdn.co/image/1e3634aa8c42ecfb6fd941d52dd7759e758d8c8d',
    u'width': 200},
   {u'height': 64,
    u'url': u'https://i.scdn.co/image/16966e323964f31b9f49c8198f874eb82a640868',
    u'width': 64}],
  u'name': u'The Go! Team',
  u'popularity': 42,
  u'type': u'artist',
  u'uri': u'spotify:artist:6veTV9sF06FBf2KN0xAdvo'},
 {u'external_urls': {u'spotify': u'https://open.spotify.com/artist/4tZwfgrHOc3mvqYlEYSvVi'},
  u'followers': {u'